# memstack-agent 上下文和配置

本 notebook 演示 Agent 的上下文管理和配置系统。

## 概述

memstack-agent 使用不可变的数据类来管理：
- **AgentContext** - 会话级上下文信息
- **ProcessorConfig** - 处理器配置选项

## 1. 导入必要模块

In [ ]:
import sys
sys.path.insert(0, '/Users/tiejunsun/github/agi-demos/src')

import json
from typing import Dict, Any

from memstack_agent import AgentContext
from memstack_agent.core.types import ProcessorConfig

## 2. AgentContext

AgentContext 包含会话级别的上下文信息，在整个会话期间保持不变。

In [ ]:
# 创建 AgentContext
context = AgentContext(
    session_id="session-001",
    conversation_id="conv-123",
    user_id="user-alice",
    project_id="project-ai-demo",
    model="claude-3-opus",
    max_tokens=200000,
    max_steps=50,
    metadata={
        "environment": "development",
        "features": ["web_search", "code_execution"]
    }
)

print("Agent Context:")
print(f"  Session ID: {context.session_id}")
print(f"  Conversation ID: {context.conversation_id}")
print(f"  User ID: {context.user_id}")
print(f"  Project ID: {context.project_id}")
print(f"  Model: {context.model}")
print(f"  Max Tokens: {context.max_tokens}")
print(f"  Max Steps: {context.max_steps}")
print(f"  Metadata: {context.metadata}")

## 3. 不可变性和 with_metadata

AgentContext 是不可变的，但可以通过 with_metadata 创建新实例。

In [ ]:
# 尝试直接修改会失败
try:
    context.user_id = "user-bob"
except Exception as e:
    print(f"Error (expected): {type(e).__name__}")

In [ ]:
# 使用 with_metadata 创建新上下文
context_v2 = context.with_metadata(
    request_id="req-001",
    timestamp="2024-01-15T10:30:00Z",
    source="api"
)

print("Original metadata:")
print(json.dumps(context.metadata, indent=2))

print("\nNew metadata:")
print(json.dumps(context_v2.metadata, indent=2))

# 验证原始上下文未被修改
print("\nOriginal unchanged:", "request_id" not in context.metadata)

## 4. ProcessorConfig

ProcessorConfig 控制处理器的行为和限制。

In [ ]:
# 创建默认配置
config = ProcessorConfig(
    model="claude-3-opus",
    temperature=0.0,
)

print("Default ProcessorConfig:")
for field in [
    "model", "api_key", "base_url", "temperature", "max_tokens",
    "max_steps", "max_tool_calls_per_step", "doom_loop_threshold",
    "max_attempts", "initial_delay_ms", "permission_timeout", 
    "continue_on_deny", "context_limit"
]:
    value = getattr(config, field)
    print(f"  {field}: {value}")

## 5. 自定义配置

In [ ]:
# 创建自定义配置
custom_config = ProcessorConfig(
    model="claude-3-sonnet",
    temperature=0.7,
    max_tokens=8192,
    max_steps=20,
    max_tool_calls_per_step=5,
    doom_loop_threshold=5,
    max_attempts=3,
    initial_delay_ms=1000,
    permission_timeout=120.0,
    continue_on_deny=True,
    context_limit=100000,
)

print("Custom ProcessorConfig:")
print(json.dumps({
    "model": custom_config.model,
    "temperature": custom_config.temperature,
    "max_tokens": custom_config.max_tokens,
    "max_steps": custom_config.max_steps,
    "max_tool_calls_per_step": custom_config.max_tool_calls_per_step,
    "doom_loop_threshold": custom_config.doom_loop_threshold,
    "max_attempts": custom_config.max_attempts,
    "initial_delay_ms": custom_config.initial_delay_ms,
    "permission_timeout": custom_config.permission_timeout,
    "continue_on_deny": custom_config.continue_on_deny,
    "context_limit": custom_config.context_limit,
}, indent=2))

## 6. 配置预设

In [ ]:
class ConfigPresets:
    """常用配置预设。"""
    
    @staticmethod
    def development() -> ProcessorConfig:
        """开发环境配置 - 更宽松的限制。"""
        return ProcessorConfig(
            model="claude-3-sonnet",
            temperature=0.5,
            max_steps=100,
            doom_loop_threshold=10,
            permission_timeout=600.0,
        )
    
    @staticmethod
    def production() -> ProcessorConfig:
        """生产环境配置 - 更严格的限制。"""
        return ProcessorConfig(
            model="claude-3-sonnet",
            temperature=0.0,
            max_steps=30,
            doom_loop_threshold=3,
            permission_timeout=60.0,
            continue_on_deny=False,
        )
    
    @staticmethod
    def creative() -> ProcessorConfig:
        """创意任务配置 - 更高的创造性。"""
        return ProcessorConfig(
            model="claude-3-opus",
            temperature=0.9,
            max_tokens=4096,
        )
    
    @staticmethod
    def coding() -> ProcessorConfig:
        """编程任务配置 - 确定性输出。"""
        return ProcessorConfig(
            model="claude-3-sonnet",
            temperature=0.0,
            max_steps=50,
            max_tool_calls_per_step=10,
        )


# 使用预设
print("Development config:")
dev = ConfigPresets.development()
print(f"  max_steps: {dev.max_steps}, temperature: {dev.temperature}")

print("\nProduction config:")
prod = ConfigPresets.production()
print(f"  max_steps: {prod.max_steps}, temperature: {prod.temperature}")

print("\nCreative config:")
creative = ConfigPresets.creative()
print(f"  temperature: {creative.temperature}")

print("\nCoding config:")
coding = ConfigPresets.coding()
print(f"  temperature: {coding.temperature}, max_tool_calls: {coding.max_tool_calls_per_step}")

## 7. with_model 方法

In [ ]:
# 使用 with_model 切换模型
base_config = ProcessorConfig(model="claude-3-sonnet")

# 切换到 opus
opus_config = base_config.with_model("claude-3-opus")

print(f"Base model: {base_config.model}")
print(f"New model: {opus_config.model}")

# 验证其他配置保持不变
print(f"\nOther settings preserved:")
print(f"  max_steps: {opus_config.max_steps}")
print(f"  temperature: {opus_config.temperature}")

## 8. 上下文验证

In [ ]:
def validate_context(context: AgentContext) -> Dict[str, Any]:
    """验证 AgentContext 的有效性。"""
    errors = []
    warnings = []
    
    # 必填字段检查
    if not context.session_id:
        errors.append("session_id is required")
    if not context.conversation_id:
        errors.append("conversation_id is required")
    if not context.user_id:
        errors.append("user_id is required")
    if not context.model:
        errors.append("model is required")
    
    # 限制检查
    if context.max_tokens < 1000:
        warnings.append(f"max_tokens ({context.max_tokens}) may be too low")
    if context.max_steps < 5:
        warnings.append(f"max_steps ({context.max_steps}) may be too low")
    
    return {
        "valid": len(errors) == 0,
        "errors": errors,
        "warnings": warnings
    }


# 测试验证
print("Valid context:")
result = validate_context(context)
print(json.dumps(result, indent=2))

# 测试无效上下文
print("\nInvalid context:")
invalid_context = AgentContext(
    session_id="",
    conversation_id="conv-123",
    user_id="",
    project_id="proj",
    model="",
    max_tokens=100,  # Too low
)
result = validate_context(invalid_context)
print(json.dumps(result, indent=2))

## 9. 配置序列化

In [ ]:
def context_to_dict(context: AgentContext) -> Dict[str, Any]:
    """将 AgentContext 转换为字典。"""
    return {
        "session_id": context.session_id,
        "conversation_id": context.conversation_id,
        "user_id": context.user_id,
        "project_id": context.project_id,
        "model": context.model,
        "max_tokens": context.max_tokens,
        "max_steps": context.max_steps,
        "metadata": context.metadata,
    }


def config_to_dict(config: ProcessorConfig) -> Dict[str, Any]:
    """将 ProcessorConfig 转换为字典。"""
    return {
        "model": config.model,
        "api_key": "***" if config.api_key else None,  # 安全处理
        "base_url": config.base_url,
        "temperature": config.temperature,
        "max_tokens": config.max_tokens,
        "max_steps": config.max_steps,
        "max_tool_calls_per_step": config.max_tool_calls_per_step,
        "doom_loop_threshold": config.doom_loop_threshold,
        "max_attempts": config.max_attempts,
        "initial_delay_ms": config.initial_delay_ms,
        "permission_timeout": config.permission_timeout,
        "continue_on_deny": config.continue_on_deny,
        "context_limit": config.context_limit,
    }


# 序列化
print("Context as dict:")
print(json.dumps(context_to_dict(context), indent=2, ensure_ascii=False))

print("\nConfig as dict:")
print(json.dumps(config_to_dict(custom_config), indent=2, ensure_ascii=False))

## 10. 完整示例：会话初始化

In [ ]:
import uuid
from datetime import datetime


def create_session(
    user_id: str,
    project_id: str,
    model: str = "claude-3-sonnet",
    environment: str = "development",
    config_preset: str = "development"
) -> tuple[AgentContext, ProcessorConfig]:
    """创建新的 Agent 会话。
    
    Args:
        user_id: 用户 ID
        project_id: 项目 ID
        model: 模型名称
        environment: 环境标识
        config_preset: 配置预设名称
    
    Returns:
        (AgentContext, ProcessorConfig) 元组
    """
    # 生成唯一标识
    session_id = f"session-{uuid.uuid4().hex[:12]}"
    conversation_id = f"conv-{uuid.uuid4().hex[:12]}"
    
    # 创建上下文
    context = AgentContext(
        session_id=session_id,
        conversation_id=conversation_id,
        user_id=user_id,
        project_id=project_id,
        model=model,
        metadata={
            "environment": environment,
            "created_at": datetime.now().isoformat(),
            "config_preset": config_preset,
        }
    )
    
    # 获取配置
    preset_map = {
        "development": ConfigPresets.development,
        "production": ConfigPresets.production,
        "creative": ConfigPresets.creative,
        "coding": ConfigPresets.coding,
    }
    
    config_fn = preset_map.get(config_preset, ConfigPresets.development)
    config = config_fn().with_model(model)
    
    return context, config


# 创建会话
ctx, cfg = create_session(
    user_id="user-alice",
    project_id="project-ai-demo",
    model="claude-3-sonnet",
    environment="development",
    config_preset="coding"
)

print("New Session Created:")
print("="*50)
print(f"Session ID: {ctx.session_id}")
print(f"Conversation ID: {ctx.conversation_id}")
print(f"User: {ctx.user_id}")
print(f"Project: {ctx.project_id}")
print(f"Model: {ctx.model}")
print(f"Config Preset: {ctx.metadata.get('config_preset')}")
print(f"Max Steps: {cfg.max_steps}")
print(f"Temperature: {cfg.temperature}")

## 总结

本 notebook 演示了 memstack-agent 的上下文和配置系统：

1. **AgentContext** - 会话级上下文信息
   - 会话和对话标识
   - 用户和项目信息
   - 自定义元数据

2. **ProcessorConfig** - 处理器配置
   - 模型和 API 设置
   - 执行限制
   - 重试和权限设置

3. **不可变性** - 所有配置都是 frozen dataclass

4. **配置预设** - 预定义的开发/生产/创意/编程配置

5. **验证和序列化** - 工具函数确保配置正确